In [ ]:
import matplotlib.pyplot as plt
import torch
from scipy.io import loadmat
import numpy as np

%load_ext autoreload
%autoreload 2
%matplotlib inline
import sys
sys.path.append('../../')

device = torch.device("mps")
#device = "cpu"
print(device, torch.__version__)
#torch.manual_seed(1802)

import torch
import torch.nn as nn
import time
from dstft import fastDSTFT
device = torch.device('mps')

def time_function(f, device, time_list=None, *args, **kwargs):
    if device.type in ['mps', 'cuda']:
        torch.cuda.synchronize() if device.type == 'cuda' else torch.mps.synchronize()
    start = time.time()
    result = f(*args, **kwargs)
    if device.type in ['mps', 'cuda']:
        torch.cuda.synchronize() if device.type == 'cuda' else torch.mps.synchronize()
    end = time.time()
    elapsed = end-start
    if time_list is None:
        print(f"Elapsed time: {elapsed:.4f} seconds")
    else:
        time_list.append(elapsed)
    return result

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
mps 2.7.0


In [ ]:
# Define x
B = 64
x = torch.randn(B, 6000, device=device)

class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.dstft = fastDSTFT(device, support=1_000, stride=50)
        self.linear = nn.Linear(501 * 101, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        print("go")
        spec = self.dstft(x)
        flat_spec = spec.view(spec.size(0), -1)
        out = self.sigmoid(self.linear(flat_spec))
        return out

# Model and optimizer
model = MyModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters: {trainable_params}")

# Dummy autograd-safe loss
dummy_loss = lambda x: x.mean()

# Warm-up
for _ in range(10):
    loss = dummy_loss(model(x))
    loss.backward()
    optimizer.zero_grad()

# Timing
num_iters = 100
start = time.time()
for _ in range(num_iters):
    for param in model.parameters():
        param.grad = None
    loss = dummy_loss(model(x))
    loss.backward()
    optimizer.step()
end = time.time()

total_time = end - start
print(f"Total time: {total_time:.4f} s")
print(f"Avg per iteration: {total_time/100:.6f} s")
print(f"Average time per iteration per sample: {total_time /num_iters /B:.6f} seconds")

def generate_bspline(self):  # cox-de boors algorithm
        c = int(self.spline_stride)
        degree = int(self.spline_degree)
        bspline = torch.ones(c, dtype=torch.float32)
        length = c

        for p in range(1, degree+1):
            length += c
            t = torch.arange(length, dtype=torch.float32)
            if p==degree:
                t *= self.spline_stride*(p+1)/(self.spline_stride*(p+1)-1)
            left = torch.zeros(length, dtype=torch.float32)
            right = torch.zeros(length, dtype=torch.float32)
            
            left[:-c] = bspline
            right[c:] = bspline.flip(0)
            bspline = (t*left + (((p+1)*c)-t) * right) / (p*c)
        return bspline

c
Total parameters: 101203
Trainable parameters: 101203
go
Elapsed time: 0.0687 seconds
go
Elapsed time: 0.0663 seconds
go
Elapsed time: 0.0741 seconds
go
Elapsed time: 0.0774 seconds
go
Elapsed time: 0.0675 seconds
go
Elapsed time: 0.0685 seconds
go
Elapsed time: 0.0699 seconds
go
Elapsed time: 0.0737 seconds
go
Elapsed time: 0.0751 seconds
go
Elapsed time: 0.0728 seconds
go
Elapsed time: 0.0739 seconds
go
Elapsed time: 0.0777 seconds
go
Elapsed time: 0.0722 seconds
go
Elapsed time: 0.0782 seconds
go
Elapsed time: 0.0686 seconds
go
Elapsed time: 0.0703 seconds
go
Elapsed time: 0.0684 seconds
go
Elapsed time: 0.0602 seconds
go
Elapsed time: 0.0725 seconds
go
Elapsed time: 0.0605 seconds
go
Elapsed time: 0.0588 seconds
go
Elapsed time: 0.0600 seconds
go
Elapsed time: 0.0594 seconds
go
Elapsed time: 0.0689 seconds
go
Elapsed time: 0.0736 seconds
go
Elapsed time: 0.0594 seconds
go
Elapsed time: 0.0595 seconds
go
Elapsed time: 0.0611 seconds
go
Elapsed time: 0.0601 seconds
go
Elapsed time: